In [1]:
import os
import base64
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

In [2]:
file = "vcard_test.vcf"
with open(file, encoding="utf8") as f:  
    data = f.readlines()

In [3]:
def get_vcards(lines) :
    all_vcards = list()
    total_cards = 0
    inside_card = False
    inside_photo = False
    photo_data = ''
    
    for line in lines:
        if ":" in line:  # parameter line 
            
            if inside_photo :  
                imgdata = base64.b64decode(photo_data)
                filename = (vcard['name']+'.jpg').replace(' ', '_') 
                with open(filename, 'wb') as f:
                     f.write(imgdata)
                vcard["photo"] = filename               
                photo_data =''
                inside_photo = False
                
            param, value = line.strip().split(':')
            
            if ("BEGIN" in param ):
                vcard = {"name":"", "org":"", "tel":"", "email":"", "photo":"" }
                inside_photo = False
                photo_data = ''
                inside_card = True
                continue
            if inside_card and ("END" in param) :
                all_vcards.append(vcard)
                inside_card = False
                total_cards +=1
                continue
            if inside_card and ("FN" in param) :
                vcard["name"] = value
                continue
            if inside_card and ("ORG" in param) :
                vcard["org"], *_ = value.split(";")
                continue
            if inside_card and ("TEL" in param) :
                if vcard.get("tel") == "" :
                    vcard["tel"] = value
                continue
            if inside_card and ("EMAIL" in param) :
                if vcard.get("email") == "" :
                    vcard["email"] = value
                continue  
            if inside_card and ("PHOTO" in param) : # ignore encoding & type parameters
                inside_photo = True
                photo_data += value
        elif inside_photo : # raw data line
            photo_data += line.strip()
            continue
        else:
            continue
        
    print("downloaded {} vcards".format(total_cards))
    return all_vcards

In [4]:
vcard_list = get_vcards(data)

downloaded 3 vcards


In [5]:
for v in vcard_list :
    print(v)

{'org': 'Organization West Company', 'photo': 'JackTest_SparrowLast.jpg', 'name': 'JackTest SparrowLast', 'email': 'jacksparrowtest@gmail.com', 'tel': '222 234 2344'}
{'org': 'Organization West Company', 'photo': 'JackTest2_SparrowLast2.jpg', 'name': 'JackTest2 SparrowLast2', 'email': 'jacksparrowtest@gmail.com', 'tel': '222 234 2344'}
{'org': 'Organization West Company', 'photo': 'JackTest3_SparrowLast3.jpg', 'name': 'JackTest3 SparrowLast3', 'email': 'jacksparrowtest@gmail.com', 'tel': '222 234 2344'}


In [6]:
X = 154
OFF = 20
y_name = OFF+10
y_org = OFF+30
y_tel = OFF+50
y_email = OFF+70
small_size = (146,196)
pic_offset = (2,2)
text_color = (0,0,0)
background_color = (255, 255, 255, 255)

for v in vcard_list :
    background = Image.new('RGBA', (350, 200), background_color ) #current size is 350x200
    draw = ImageDraw.Draw(background)
    
    font = ImageFont.truetype("Roboto-Regular.ttf", 12) # font = ImageFont.truetype(<font-file>, <font-size>)
    
    draw.text((X, y_name),v['name'], text_color, font=font) # draw.text((x, y),"Sample Text",(r,g,b))
    draw.text((X, y_org),v['org'], text_color, font=font)
    draw.text((X, y_tel),v['tel'], text_color, font=font)
    draw.text((X, y_email),v['email'], text_color, font=font)
    
    if v['photo'] != '' :
        img = Image.open(v['photo'], 'r')
        small_img = img.resize(small_size)
        background.paste(small_img, pic_offset)
        # os.remove(v['photo'])   # we can remove jpg files with original pictures from vcards
        
    background.save( (v['name']+'.png').replace(' ', '_') )
   